## 写成RV汇编代码
```
loop(int*, int):                             # @loop(a->a0: int*, s->a1: int)
        addi    a0, a0, 2047
        addi    a0, a0, 1949
        li      a2, -1000
.LBB0_1:                                # =>This Inner Loop Header: Depth=1
        lw      a3, 0(a0)
        add     a3, a3, a1
        sw      a3, 0(a0)
        addi    a2, a2, 1
        addi    a0, a0, -4
        bnez    a2, .LBB0_1
```